Goals:
1. Create a corpus of all yt video descriptions for scam videos in the data.
2. vectorize this.
3. Use this to create a regressor capturing the degree to which a video description is "spam-speaky" according to ML result (or dichotomous 1-spammy, 0-notSpammy).
4.Use this bag-of-words model to generate values for a new variable, "spam-speakiness", for every observation in the dataset .
5. (optional) repeat this process with a dataset of spam-lookalike videos which warn AGAINST scams and spam and attempt to create another regressor, "virtue-speakiness".
6. get "has_numbers" and "number_sum" for numbers in character vars
7. multiply-impute missing values b4 machine learning (training data only, drop missing on test!!!!)
8. get capital letter frequency
9. get emoji frequency

In [237]:
import os
import json 
import pandas as pd
import pymongo
#from google.colab import drive
#nltk.download('stopwords')
#nltk.download('averaged_perceptron_tagger')
from pymongo import MongoClient
import socket
import urllib.request as urllib2
import pandas as pd
import numpy as np
import nltk  
import numpy as np  
import random  
import string
import regex
import urllib.request  
import re  
from string import punctuation
from os import listdir
from collections import Counter
from nltk.corpus import stopwords
import math
from nltk.tokenize.casual import TweetTokenizer
import enchant
import re
import probablepeople as pp
from nltk.tokenize import sent_tokenize
t = TweetTokenizer()
import emoji
import heapq
from collections.abc import Iterable



<h1>Part 1: PUll existing data from DB</h1>

In [77]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)


db = client.videos_mdb
collection = db["videos"]  

In [189]:
#pull data from mongo (just one iv for now)
linVarList=["id","spam","description","title","commentSection"]
df=pd.DataFrame(columns=linVarList)
row=[]
counter=0
data = collection.find({})
for item in data:
    for var in linVarList:
        if var in item:
            row.append(item[var])
        else:
            row.append(np.nan)
    df.loc[counter] = row
    counter+=1
    row=[]

In [186]:
#Initial pass cleaning data- make into separate sentences
def make_sentences(x):
    data = x.replace("\n",".")
    data=nltk.sent_tokenize(data)
    return(data)

In [235]:
#Function to flatten simple lists
def flatten_it(item):
    flat=[x for l in item for x in l] 
    return(flat)

In [236]:
#Function to flatten complex lists

def flatten(l):
    for el in l:
        if isinstance(el, Iterable) and not isinstance(el, (str, bytes)):
            yield from flatten(el)
        else:
            yield el


In [244]:
#Function to flatten and join each comment section into one string:
def join_comments(row):
    almost_flat =list(flatten(row))
    actually_flat=" ".join(almost_flat)
    return(actually_flat)

In [246]:
#New DF Columns:
#Get tokenized sentences for each description and title in df
df["sentTokenDescript"] =df["description"].apply(lambda row : make_sentences(row))
df["sentTokenTitle"] =df["title"].apply(lambda row : make_sentences(row))
df["commentSectionString"] =df["commentSection"].apply(lambda row: join_comments(row))
df["sentTokenComments"] =df["commentSectionString"].apply(lambda row: make_sentences(row))


In [247]:
df["sentTokenComments"]

0      [Subscribers badhe kya Taarak Mehta Ka Ooltah ...
1      [🔥The #1 &amp; Most Easiest Way To Make Money ...
2      [Thanks for more guidance and lessons<br>plus,...
3      [smash that like button!, Obliterate that like...
4      [Thanks for more guidance and lessons<br>PULS,...
                             ...                        
195    [What happened to this guy Thank you so much f...
196    [🔔 Subscribe to Get the Latest Tips, Tricks, a...
197    [🔔 Subscribe to Get the Latest Tips, Tricks, a...
198    [🔔 Subscribe to Get the Latest Tips, Tricks, a...
199    [🔔 Subscribe to Get the Latest Tips, Tricks, a...
Name: sentTokenComments, Length: 200, dtype: object

In [187]:
#Function for cleaning and Tokenizing all text
def clean_text(text):
    tokenlist=[]
    tokens=t.tokenize(text)
    for token in tokens:
        #get rid of /n and similar chars:
        data = regex.findall(r'\X', token)
        if any(char for char in data)>1:
            token = token.replace(char,"")
        if not token.isalpha():
            if token in ["!","?","@","#"] or token in emoji.UNICODE_EMOJI['en']:
                tokenlist.append(token)
            else:
                continue
        else:
            if len(token)>1:
                tokenlist.append(token.lower())
        #get rid of selective puntuation, leaving in emojis, #, ?, and !
        #replace any links with "includes_a_link", treating any link as the same word
            
    return(tokenlist)

In [130]:
def getwords(sentenceList):
    output=[]
    for sentence in sentenceList:
        mystring= str(sentence)
        clean=clean_text(mystring)
        output.append(clean)
    return(output)

In [248]:
#New DF Column
# Get a list of lists of words in each sentence in descriptions, titles, and comments
df["wordTokenDescript"] =df["sentTokenDescript"].apply(lambda row : getwords(row))
df["wordTokenTitle"] =df["sentTokenTitle"].apply(lambda row : getwords(row))
df["wordTokenComments"] =df["sentTokenComments"].apply(lambda row : getwords(row))

In [157]:
def get_freq(tokenlist):
    wordfreq = {}
    for token in tokenlist:
        if token.lower() not in wordfreq:
            wordfreq[token.lower()] = 1
        else:
            wordfreq[token.lower()] += 1
    return(wordfreq)


In [253]:
#New DF Column
# Get parts of speech for every word in description, then flatten the list of sentences
df["POS_Descript"]=df["wordTokenDescript"].apply(lambda row: flatten_it([nltk.pos_tag(i) for i in row]))
df["POS_Title"]=df["wordTokenTitle"].apply(lambda row: flatten_it([nltk.pos_tag(i) for i in row]))
df["POS_Comments"]=df["wordTokenComments"].apply(lambda row: flatten_it([nltk.pos_tag(i) for i in row]))

In [255]:
#New DF Column
#Get word frequencies for whole description, also flattening it first
df["wordFreqDescript"]=df["wordTokenDescript"].apply(lambda row: get_freq(flatten_it(row)))
df["wordFreqTitle"]=df["wordTokenTitle"].apply(lambda row: get_freq(flatten_it(row)))
df["wordFreqComments"]=df["wordTokenComments"].apply(lambda row: get_freq(flatten_it(row)))

In [252]:
df.to_csv("tokenizedDf.csv")

In [271]:
#Create training data from X percent of the total entries

y=df["spam"]
x=pd.DataFrame()
for item in df:
    if item !="spam":
        x[item]=df[item]

X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.33, random_state=42)

In [272]:
bag_train=xtrain.loc[df["spam"]==1]

In [273]:
rawDescriptions=[(item for item in bag_train["description"])]
corpus=""
for item in rawDescriptions:
    corpus=corpus.join(item)
    

In [ ]:
wordfreq = {}


for token in tokens:
        wordfreq[token.lower()] = 1
    else:
        wordfreq[token.lower()] += 1

exclaim=0
question=0
hashy=0
atme=0

for item in corpus:
    if item=="!":
        exclaim+=1
    elif item == "?":
        question+=1
    elif item =="#":
        hashy+=1


wordfreq["?"]=question
wordfreq["!"]=exclaim



In [ ]:
import heapq

sorted_freq=sorted(wordfreq.items(), key=lambda x: x[1], reverse=True)
most_freq=sorted_freq[:200]
#most_freq = [i for i in wordfreq if wordfreq[i] >= 1 and i[0:4]!="http" ]
#wordfreqmost_freq = heapq.nlargest(200, wordfreq, key=wordfreq.get)

In [ ]:

sorted_freq=sorted(wordfreq.items(), key=lambda x: x[1], reverse=True)
most_freq=sorted_freq[:200]

In [ ]:
boringWords=["with","that","this","will","your","there", "wesleyvirgin", "wesleymilliondollarvirgin","just", "they", "much","kaise","then", "than", "when", "here","these", "girl", "what","where","from", "wesley", "virgin", "also",  "about", "them", "which", "dave", "very"] #"girl" is removed b/c comes from name of big scam youtuber and leaving it in risks sexist codings

In [ ]:
#number of words in spamwords/ total number of words = regressor
sorted_freq

[('!', 76),
 ('?', 19),
 ('video', 2),
 ('ryan', 1),
 ('hildreth', 1),
 ('shows', 1),
 ('get', 1),
 ('free', 1),
 ('100', 1),
 ('paypal', 1),
 ('gift', 1),
 ('cards', 1),
 ('everyday', 1),
 ('autopilot', 1),
 ('easy', 1),
 ('money', 1),
 ('my', 1),
 ('1', 1),
 ('recommendation', 1),
 ('for', 1),
 ('making', 1),
 ('online', 1),
 ('httpswwwfunnelfromhomecomearn500', 1),
 ('if', 1),
 ('’', 1),
 ('know', 1),
 ('i', 1),
 ('name', 1),
 ('7', 1),
 ('figure', 1),
 ('digital', 1),
 ('marketer', 1),
 ('created', 1),
 ('multiple', 1),
 ('businesses', 1),
 ('beginning', 1),
 ('january', 1),
 ('2017', 1),
 ('allowed', 1),
 ('quit', 1),
 ('boring', 1),
 ('9', 1),
 ('5', 1),
 ('job', 1),
 ('time', 1),
 ('spend', 1),
 ('family', 1),
 ('friends', 1),
 ('travel', 1),
 ('world', 1),
 ('buy', 1),
 ('dream', 1),
 ('cars', 1),
 ('invest', 1),
 ('real', 1),
 ('estate', 1),
 ('marry', 1),
 ('woman', 1),
 ('things', 1),
 ('sound', 1),
 ('like', 1),
 ('something', 1),
 ('striving', 1),
 ('please', 1),
 ('subscr